In [ ]:
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os
import numpy as np

from Utils import loader, loss
from Train import train
from Model import Model

In [ ]:
imdir_rs19 = '/home/physik/Dokumente/rs19/jpgs/rs19_val'
maskdir_rs19 = '/home/physik/Dokumente/rs19/uint8/rs19_val'
#imdir_rs19 = '/home/luis/Documents/ImSeg/rs19_val/jpgs/rs19_val'
#maskdir_rs19 = '/home/luis/Documents/ImSeg/rs19_val/uint8/rs19_val'
classes_ = {"rail-track","tram-track"}#,"rail-raised","rail-embedded"}
range_train = (0,4999)
range_test = (5000,7499)
range_val = (7500,8499)
img_size = (1024,512)

In [ ]:
traindata = loader.dataset(imdir=imdir_rs19,
                      maskdir=maskdir_rs19,
                      numstart=range_train[0],
                      numend=range_train[1],
                      classes=classes_,
                      img_size = img_size)

testdata = loader.dataset(imdir=imdir_rs19,
                      maskdir=maskdir_rs19,
                      numstart=range_test[0],
                      numend=range_test[1],
                      classes=classes_,
                      img_size = img_size)

valdata = loader.dataset(imdir=imdir_rs19,
                      maskdir=maskdir_rs19,
                      numstart=range_val[0],
                      numend=range_val[1],
                      classes=classes_,
                      img_size = img_size)

In [ ]:
model_path = "/home/physik/Model_19122022_2"
#model_path = "/home/luis/Documents/HSD_ARBEIT/Model_19122022_2"

In [ ]:
model = Model.UNet(num_classes=3,p_dropout=0.3)

In [ ]:
model = model.to('cuda')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.load_state_dict(torch.load(model_path,map_location=torch.device(device)))

In [ ]:
lossf=loss.BCEDiceLoss(lambda_dice=0.4,lambda_bce=0.6)

In [ ]:
from Attacks import attacks

In [ ]:
#perturbed_testimg = attacks.I_FGSM_singleImage(model,
#                         lossf,
#                         valdata[44][0],
#                         valdata[9][1],
#                         0.00001,10000)

In [ ]:
#valdata[44][0][None].shape

In [ ]:
perturbed_testbatch = attacks.PGD_batch(model=model, 
         lossf=lossf,
         img_batch=valdata[44][0][None], 
         mask_batch=valdata[44][1][None],
         target_mask_batch=valdata[9][1][None], 
         num_iters=200,
         step_norm=2,
         step_size=7.0,
         eps=10.0,
         eps_norm=2,
         clamp=(0,1))

In [ ]:
torch.save(perturbed_testbatch,'perturbed_testbatch.pkl')

In [ ]:
perturbed_testbatch = torch.load('perturbed_testbatch.pkl')

In [ ]:
#perturbed_testimg = attacks.I_FGSMLeastLikely_singleImage(model,
#                         lossf,
#                         valdata[38][0],
#                         3,
#                         0.005,500)

In [ ]:
fig = plt.figure(figsize=(512/30,1024/30))
plt.imshow(valdata[44][0].cpu().detach().numpy().transpose(1,2,0))
plt.show()

In [ ]:
fig = plt.figure(figsize=(512/30,1024/30))
plt.imshow(perturbed_testbatch[0].cpu().detach().numpy().transpose(1,2,0))
plt.show()

In [ ]:
plt.imshow(valdata[9][0].cpu().detach().numpy().transpose(1,2,0))

In [ ]:
plt.imshow(valdata[9][1][2].cpu().detach().numpy())

In [ ]:
d_device = 'cuda' if torch.cuda.is_available() else 'cpu'
#d_device = 'cpu'
prediction = model.predict(valdata[44][0].unsqueeze(0).to(d_device))

In [ ]:
plt.imshow(prediction[1][0][2].cpu().detach().numpy())

In [ ]:
prediction_adv = model.predict(perturbed_testbatch.to(d_device))

In [ ]:
plt.imshow(prediction_adv[1][0][2].cpu().detach().numpy())

In [ ]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
report_gpu()

In [ ]:
print(torch.cuda.list_gpu_processes())

In [ ]:
!pip install pynvml